# Mock Code for BEES Analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Read in datasets

In [ ]:
# Need to enter filename for enstar data here
enstar_data = pd.read_csv()

bees_records = pd.read_csv("bees_records_with_GoogleMapsAPI_standardized_addresses.csv")

matsu_property = pd.read_csv("matsu_borough_residential_property_tax_data.csv")

anc_com_property = pd.read_csv("anchorage_property_tax_data_commercial_housing_side.csv")

anc_res_property = pd.read_csv("anchorage_property_tax_data_residential_side.csv")

kenai_property = pd.read_csv("kenai_peninsula_borough_property_tax_data.csv")

## Join datasets

### Enstar, BEES, Property Tax datasets

In [ ]:
# I don't have the column headers for the Enstar data, so this will need to be added
enstar_bees = pd.merge(enstar_data, bees_records, left_on='enter_enstar_address_field_here', 
                         right_on='final_address', how='left')


# The final_site_address field is the one I gave for you guys to convert using the GoogleMapsAPI Geocoder.
# If you created a new column name for the standardized address, it should be entered instead.
# Also, let me know if you need me to convert these addresses using the Geocoder.
final_dataset = pd.merge(enstar_bees, matsu_property, left_on='enter_enstar_address_field_here',
                        right_on='final_site_address', how='left')


# The final_site_address field is the one I gave for you guys to convert using the GoogleMapsAPI Geocoder.
# If you created a new column name for the standardized address, it should be entered instead.
# Also, let me know if you need me to convert these addresses using the Geocoder.
final_dataset = pd.merge(final_dataset, anc_com_property, left_on='enter_enstar_address_field_here',
                        right_on='final_site_address', how='left')


# The final_site_address field is the one I gave for you guys to convert using the GoogleMapsAPI Geocoder.
# If you created a new column name for the standardized address, it should be entered instead.
# Also, let me know if you need me to convert these addresses using the Geocoder.
final_dataset = pd.merge(final_dataset, anc_res_property, left_on='enter_enstar_address_field_here',
                        right_on='final_site_address', how='left')


# The final_site_address field is the one I gave for you guys to convert using the GoogleMapsAPI Geocoder.
# If you created a new column name for the standardized address, it should be entered instead.
# Also, let me know if you need me to convert these addresses using the Geocoder.
final_dataset = pd.merge(final_dataset, kenai_property, left_on='enter_enstar_address_field_here',
                        right_on='final_site_address', how='left')

### Heating Degree Day Database

In [ ]:
# I put these folders in the Google Drive folder "shared_datasets_with_marco"
# We will need this data to be able to normalize for the climate in different 
# years. 

# This dataset allows us to look up the best weather station for heating degree
# data based on the zip code.
zipcode_to_weather_station = pd.read_csv("all_alaska_zipcode_to_station_lookups.csv")

# You will need to create a zip code column from the address field in the final dataset
final_dataset_with_zips = pd.merge(final_dataset, zipcode_to_weather_station, how='left',
                                      left_on='create_zipcode_field_here', right_on='zip_code')

# This dataset has the monthly heating degree days from all of the reliable 
# weather stations in Alaska that have data going back 10 years.  We may need
# to get some additional data that goes back farther in time.
hdd_data = pd.read_csv("all_alaska_stations_base_60and65_hdd.csv")

# This will merge the correct heating degree day data for each zip code
# using the proper weather station, year, and month.  You will have to 
# create two columns from the ENSTAR timestamp data for this to work:
# the 'year_field' and the 'month_field'
final_dataset_with_hdds = pd.merge(final_dataset_with_zips, hdd_data, how='left',
                                  left_on=['station_name', 'year_field', 'month_field'], 
                                   right_on=['station_name', 'year', 'month'])

## Normalize Data

In [ ]:
''' The next step is to take the monthly energy consumptions and divide it by the 
monthly_base_60_hdds. I would probably first multiply the ccf of natural gas by 100,000 to convert 
it to BTUs.  After that we should normalize by the square footage of the building, taken from the
AkWarm BEES ratings or from the property tax databases. The final measure we will have is sometimes 
known as the "Home Heating Index", and the units will be BTUs/HDD/square foot.'''